# Möbius Plateau Auto‑Explorer Ω  

**Grandma‑friendly version – paste your scan rows below, click Runtime ▶ Run all.**  

What it does in *one* pass  

1. Reads your fine‑scan table (`bits, k, φ, λ, rel_err`).  
2. Tags rows into plateaux at three granularities (ε = 1e‑4, 5e‑6, 1e‑5).  
3. Applies a **mega bank of ops**  

```
×9 ÷9 ×φ ÷φ ×φ² ÷φ² ×φ½ ÷φ½  
×α ÷α ×α² ÷α² ×α½ ÷α½  
×δ ÷δ ×δ² ÷δ² ×δ½ ÷δ½  
×Δ ÷Δ ×Δ² ÷Δ² ×Δ½ ÷Δ½  
×(1‑c) ×c⁻¹   (for every c above)  
```  

(φ golden ratio, α fine‑structure, δ Feigenbaum δ≈4.669, Δ Feigenbaum α≈2.5029).  

4. Prints the first 60 rows for **every (ε,op)** so you eyeball structure.  
5. Saves two Excel files in *Content*:

* `plateaus_raw.xlsx` – untouched rows  
* `plateaus_ops.xlsx` – one sheet per (ε, op)  

No widgets, no parameters – just run.  


In [1]:
import pandas as pd, numpy as np
from itertools import product

# 👉 Paste / update your raw rows below -------------------
raw_rows = [
    # bits, k, φ‑micro, λ, rel_err
    (64,  3.6, 0.003942, 0.0, 0.4117525875906678269),
    (64,  3.8, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.0, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.2, 0.003942, 0.0, 0.4117525875906678269),
    (128, 3.6, 0.003942, 0.0, 0.4117525875906678271),
    (128, 3.8, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.0, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.2, 0.003942, 0.0, 0.4117525875906678271),
]
df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
print("Preview →")
display(df.head())


Preview →


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,3.6,0.003942,0.0,0.411753


In [2]:
# === built‑in constants ===
phi = (1+5**0.5)/2          # 1.61803398875…
phi_inv = 1/phi
alpha_inv = 137.035999084    # fine structure 1/α
alpha = 1/alpha_inv
delta = 4.669201609          # Feigenbaum δ
alpha_F = 2.502907875        # Feigenbaum α
# helper inverses and complements
def comp(c): return 1-c
consts = {
    'φ':phi,'φ²':phi**2,'φ½':phi**0.5,'φ⁻¹':phi_inv,'φ⁻²':phi_inv**2,'φ⁻½':phi_inv**0.5,
    'α':alpha,'α²':alpha**2,'α½':alpha**0.5,'α⁻¹':alpha_inv,'α⁻²':alpha_inv**2,'α⁻½':alpha_inv**0.5,
    'δ':delta,'δ²':delta**2,'δ½':delta**0.5,'δ⁻¹':1/delta,'δ⁻²':1/delta**2,'δ⁻½':delta**-0.5,
    'Δ':alpha_F,'Δ²':alpha_F**2,'Δ½':alpha_F**0.5,'Δ⁻¹':1/alpha_F,'Δ⁻²':1/alpha_F**2,'Δ⁻½':alpha_F**-0.5,
}

# core op lambdas
OPS = {
    '×9':  lambda x: x*9,   '÷9':  lambda x: x/9,
}
# extend with all constants multiplications / inverses / complements and powers
for name,val in consts.items():
    OPS[f'×{name}'] = lambda x,val=val: x*val
    OPS[f'÷{name}'] = lambda x,val=val: x/val
    OPS[f'+{name}'] = lambda x,val=val: x+val
    OPS[f'-{name}'] = lambda x,val=val: x-val
# you can add more OPS here if desired
print(f"Loaded {len(OPS)} ops")


Loaded 98 ops


In [3]:
def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    out = frame.copy()
    out['plateau'] = ((out['rel_err']-base)/eps).astype(int)
    return out

views = []
for eps in [1e-4,5e-5,1e-5]:
    tagged = tag_plateaus(df, eps)
    for op_name, op_fun in OPS.items():
        tmp = tagged.copy()
        tmp['ε'] = eps
        tmp['op'] = op_name
        tmp['rel_err'] = op_fun(tmp['rel_err'])
        views.append(tmp.iloc[:60])   # first 60 lines peek

out = pd.concat(views, ignore_index=True)
afile = 'plateaus_raw.xlsx'
bfile = 'plateaus_ops.xlsx'
df.to_excel(afile,index=False)
out.to_excel(bfile,index=False)
print(f"Wrote {afile} and {bfile}")
display(out.head(30))


Wrote plateaus_raw.xlsx and plateaus_ops.xlsx


,bits,k,phi,lam,rel_err,plateau,ε,op
0,64,3.6,0.003942,0.0,3.705773,0,0.0001,×9
1,64,3.8,0.003942,0.0,3.705773,0,0.0001,×9
2,64,4.0,0.003942,0.0,3.705773,0,0.0001,×9
3,64,4.2,0.003942,0.0,3.705773,0,0.0001,×9
4,128,3.6,0.003942,0.0,3.705773,0,0.0001,×9
5,128,3.8,0.003942,0.0,3.705773,0,0.0001,×9
6,128,4.0,0.003942,0.0,3.705773,0,0.0001,×9
7,128,4.2,0.003942,0.0,3.705773,0,0.0001,×9
8,64,3.6,0.003942,0.0,0.045750,0,0.0001,÷9
9,64,3.8,0.003942,0.0,0.045750,0,0.0001,÷9
